In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [3]:
con = sqlite3.connect("laliga.sqlite")
df = pd.read_sql_query("SELECT * from Matches", con)

In [4]:
def get_result(score: str):
    if score is None:
        return None
    goals = list(map(int, score.split(':')))
    if goals[0]>goals[1]:
        return "1"
    elif goals[1]>goals[0]:
        return "2"
    else:
        return "X"

In [5]:
df['result'] = df['score'].apply(get_result)
df['result'] = df['result'].map(str)

In [7]:
def get_goals(score: str, home_away: int):
    if score is None:
        return None
    goals = list(map(int, score.split(':')))
    return goals[home_away]

In [8]:
df['home_goals'] = df['score'].apply(get_goals, args=(0,))
df['away_goals'] = df['score'].apply(get_goals, args=(1,))
df.head()

,season,division,matchday,date,time,home_team,away_team,score,result,home_goals,away_goals
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,2,2.0,3.0
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,1,3.0,2.0
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,1,5.0,0.0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,X,1.0,1.0
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,2,0.0,2.0


In [9]:
df_past = df.loc[df['season'] != '2021-2022'].copy()

In [10]:
df_past.dtypes

season         object
division        int64
matchday        int64
date           object
time           object
home_team      object
away_team      object
score          object
result         object
home_goals    float64
away_goals    float64
dtype: object

In [26]:
dfs = []
for season in df_past['season'].drop_duplicates():
    for division in df_past.loc[(df_past['season']==season), 'division'].drop_duplicates():
        df_games = df_past.loc[(df_past['season']==season) & (df_past['division']==division)]
        teams = df_games['home_team'].drop_duplicates().rename('team')
        init_data = [(season, division, 0, 0, 0, 0, 0) for _ in teams]
        df_standings = pd.DataFrame(init_data, columns=['season', 'division', 'matchday', 'GF', 'GA', 'GD', 'Pts'], index=teams)
        for matchday in df_games['matchday'].drop_duplicates():
            df_standings['matchday'] += 1
            df_matchday = df_games.loc[df_games['matchday']==matchday]
            
            for i in df_matchday.index:
                game = df_matchday.loc[i, :]
                df_standings.loc[game['home_team'], 'GF'] += game['home_goals']
                df_standings.loc[game['home_team'], 'GA'] += game['away_goals']
                
                if game['result'] == '1':
                    
                    df_standings.loc[game['home_team'], 'Pts'] += 3
                    
                elif game['result'] == '2':
                    
                    pass
                else:
                    
                    df_standings.loc[game['home_team'], 'Pts'] += 1
                    
            df_standings['GD'] = (df_standings['GF'] - df_standings['GA']).astype(int)
            df_standings.sort_values(by=['Pts', 'GD', 'GF'], ascending=False, inplace=True)
            df_standings.reset_index(inplace=True)
            df_standings.insert(value=np.arange(1, len(df_standings)+1), loc=3, column='rank')
            dfs.append(df_standings[['season', 'division', 'matchday', 'rank', 'team', 'GF', 'GA', 'GD', 'Pts']].copy())
            df_standings.drop(columns=['rank'], inplace=True)
            df_standings.set_index(keys='team', drop=True, inplace=True)
all_home = pd.concat(dfs, ignore_index=True)

            

In [28]:
dfs = []
for season in df_past['season'].drop_duplicates():
    for division in df_past.loc[(df_past['season']==season), 'division'].drop_duplicates():
        df_games = df_past.loc[(df_past['season']==season) & (df_past['division']==division)]
        teams = df_games['home_team'].drop_duplicates().rename('team')
        init_data = [(season, division, 0, 0, 0, 0, 0) for _ in teams]
        df_standings = pd.DataFrame(init_data, columns=['season', 'division', 'matchday', 'GF', 'GA', 'GD', 'Pts'], index=teams)
        for matchday in df_games['matchday'].drop_duplicates():
            df_standings['matchday'] += 1
            df_matchday = df_games.loc[df_games['matchday']==matchday]
            
            for i in df_matchday.index:
                game = df_matchday.loc[i, :]
                df_standings.loc[game['away_team'], 'GF'] += game['away_goals']
                df_standings.loc[game['away_team'], 'GA'] += game['home_goals']
                
                if game['result'] == '1':
                    
                    pass
                    
                elif game['result'] == '2':
                    
                    df_standings.loc[game['away_team'], 'Pts'] += 3
                else:
                    
                    df_standings.loc[game['away_team'], 'Pts'] += 1
                    
            df_standings['GD'] = (df_standings['GF'] - df_standings['GA']).astype(int)
            df_standings.sort_values(by=['Pts', 'GD', 'GF'], ascending=False, inplace=True)
            df_standings.reset_index(inplace=True)
            df_standings.insert(value=np.arange(1, len(df_standings)+1), loc=3, column='rank')
            dfs.append(df_standings[['season', 'division', 'matchday', 'rank', 'team', 'GF', 'GA', 'GD', 'Pts']].copy())
            df_standings.drop(columns=['rank'], inplace=True)
            df_standings.set_index(keys='team', drop=True, inplace=True)
all_away = pd.concat(dfs, ignore_index=True)


In [29]:
features = pd.read_csv('feature_frame.csv')
features.tail()

,Unnamed: 0,season,division,matchday,date,time,home_team,away_team,score,home_goals,...,result,home_rank,away_rank,home_form,away_form,home_GF_pg,home_GA_pg,away_GF_pg,away_GA_pg,year_time
0,0,1928-1929,1,1,2/10/29,NaN,Arenas Club,Athletic Madrid,2:3,2,...,2,1,1,0,0,0.0,0.0,0.0,0.0,0
1,1,1928-1929,1,1,2/10/29,NaN,Espanyol,Real Unión,3:2,3,...,1,1,1,0,0,0.0,0.0,0.0,0.0,0
2,2,1928-1929,1,1,2/10/29,NaN,Real Madrid,Catalunya,5:0,5,...,1,1,1,0,0,0.0,0.0,0.0,0.0,0
3,3,1928-1929,1,1,2/10/29,NaN,Donostia,Athletic,1:1,1,...,X,1,1,0,0,0.0,0.0,0.0,0.0,0
4,4,1928-1929,1,1,2/12/29,NaN,Racing,Barcelona,0:2,0,...,2,1,1,0,0,0.0,0.0,0.0,0.0,0


In [31]:
all_home.head(20)

,season,division,matchday,rank,team,GF,GA,GD,Pts
0,1928-1929,1,1,1,Real Madrid,5.0,0.0,5,3
1,1928-1929,1,1,2,Espanyol,3.0,2.0,1,3
2,1928-1929,1,1,3,Donostia,1.0,1.0,0,1
3,1928-1929,1,1,4,Barcelona,0.0,0.0,0,0
4,1928-1929,1,1,5,Athletic,0.0,0.0,0,0
5,1928-1929,1,1,6,Athletic Madrid,0.0,0.0,0,0
6,1928-1929,1,1,7,Real Unión,0.0,0.0,0,0
7,1928-1929,1,1,8,Catalunya,0.0,0.0,0,0
8,1928-1929,1,1,9,Arenas Club,2.0,3.0,-1,0
9,1928-1929,1,1,10,Racing,0.0,2.0,-2,0


In [38]:
def get_home_rank(row):
    if row.matchday == 1:
        return 1
    else:
        frame = all_home[(all_home.team == row.home_team)&(all_home.season == row.season)&(all_home.matchday == row.matchday -1)]
        rank = frame.iloc[0,3]
        return rank
features['home_rank_HT'] = features.apply(get_home_rank , axis = 1)  

In [39]:
def get_away_rank(row):
    if row.matchday == 1:
        return 1
    else:
        frame = all_away[(all_away.team == row.away_team)&(all_away.season == row.season)&(all_away.matchday == row.matchday -1)]
        rank = frame.iloc[0,3]
        return rank
features['away_rank_AT'] = features.apply(get_away_rank , axis = 1)  

In [63]:
def last_confronts(row):
    a = row['season']
    dt = df_past[(df_past.home_team == row.home_team) & (df_past.away_team == row.away_team)& (df_past.season < a)].tail(3)
    res = dt[['result']].values.tolist()
    a = res.count(['1']) - res.count(['2'])
    return a


In [41]:
features.head(20)

,Unnamed: 0,season,division,matchday,date,time,home_team,away_team,score,home_goals,...,away_rank,home_form,away_form,home_GF_pg,home_GA_pg,away_GF_pg,away_GA_pg,year_time,home_rank_HT,away_rank_AT
0,0,1928-1929,1,1,2/10/29,NaN,Arenas Club,Athletic Madrid,2:3,2,...,1,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1
1,1,1928-1929,1,1,2/10/29,NaN,Espanyol,Real Unión,3:2,3,...,1,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1
2,2,1928-1929,1,1,2/10/29,NaN,Real Madrid,Catalunya,5:0,5,...,1,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1
3,3,1928-1929,1,1,2/10/29,NaN,Donostia,Athletic,1:1,1,...,1,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1
4,4,1928-1929,1,1,2/12/29,NaN,Racing,Barcelona,0:2,0,...,1,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1
5,5,1928-1929,1,2,2/17/29,NaN,Barcelona,Real Madrid,1:2,1,...,1,1,1,2.000000,0.000000,5.000000,0.000000,0,4,6
6,6,1928-1929,1,2,2/17/29,NaN,Athletic,Espanyol,9:0,9,...,3,0,1,1.000000,1.000000,3.000000,2.000000,0,5,5
7,7,1928-1929,1,2,2/17/29,NaN,Athletic Madrid,Donostia,0:3,0,...,5,1,0,3.000000,2.000000,1.000000,1.000000,0,6,7
8,8,1928-1929,1,2,2/17/29,NaN,Real Unión,Racing,3:1,3,...,9,-1,-1,2.000000,3.000000,0.000000,2.000000,0,7,8
9,9,1928-1929,1,2,2/17/29,NaN,Catalunya,Arenas Club,5:2,5,...,7,-1,-1,0.000000,5.000000,2.000000,3.000000,0,8,4


In [54]:
last_confronts(features_2.iloc[5,:])

0

In [44]:
new = df_past['season'].str.split('-', n = 1, expand = True)
new[0] = new[0].astype('int')
new[1] = new[1].astype('int')


In [45]:
df_past['season'] = new[0]

In [46]:
features_2 = features.copy()

In [47]:
features_2['season'] = new[0]

In [52]:
features_2.iloc[1,1]

1928

In [64]:
features_2['last_conf'] = features_2.apply(last_confronts, axis = 1)   

In [65]:
features_2['last_conf'].value_counts()

 0    15578
 1    11824
 2     7900
 3     6003
-1     4888
-2     1516
-3      469
Name: last_conf, dtype: int64

In [58]:
features_2.dtypes

Unnamed: 0        int64
season            int32
division          int64
matchday          int64
date             object
time             object
home_team        object
away_team        object
score            object
home_goals        int64
away_goals        int64
result           object
home_rank         int64
away_rank         int64
home_form         int64
away_form         int64
home_GF_pg      float64
home_GA_pg      float64
away_GF_pg      float64
away_GA_pg      float64
year_time         int64
home_rank_HT      int64
away_rank_AT      int64
last_conf         int64
dtype: object

In [59]:
dt = df_past[(df_past.home_team == 'Barcelona') & (df_past.away_team == 'Real Madrid')& (df_past.season < 1995)].tail(3)
s = dt[[result]].values.tolist()

,season,division,matchday,date,time,home_team,away_team,score,result,home_goals,away_goals
14875,1992,1,1,9/5/92,None,Barcelona,Real Madrid,2:1,1,2.0,1.0
15427,1993,1,18,1/9/94,None,Barcelona,Real Madrid,5:0,1,5.0,0.0
15978,1994,1,35,5/27/95,None,Barcelona,Real Madrid,1:0,1,1.0,0.0


In [61]:
s = dt[['result']].values.tolist()
s

[['1'], ['1'], ['1']]

In [62]:
s.count(['1'])

3

In [66]:
features_2.head(20)

,Unnamed: 0,season,division,matchday,date,time,home_team,away_team,score,home_goals,...,home_form,away_form,home_GF_pg,home_GA_pg,away_GF_pg,away_GA_pg,year_time,home_rank_HT,away_rank_AT,last_conf
0,0,1928,1,1,2/10/29,NaN,Arenas Club,Athletic Madrid,2:3,2,...,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1,0
1,1,1928,1,1,2/10/29,NaN,Espanyol,Real Unión,3:2,3,...,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1,0
2,2,1928,1,1,2/10/29,NaN,Real Madrid,Catalunya,5:0,5,...,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1,0
3,3,1928,1,1,2/10/29,NaN,Donostia,Athletic,1:1,1,...,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1,0
4,4,1928,1,1,2/12/29,NaN,Racing,Barcelona,0:2,0,...,0,0,0.000000,0.000000,0.000000,0.000000,0,1,1,0
5,5,1928,1,2,2/17/29,NaN,Barcelona,Real Madrid,1:2,1,...,1,1,2.000000,0.000000,5.000000,0.000000,0,4,6,0
6,6,1928,1,2,2/17/29,NaN,Athletic,Espanyol,9:0,9,...,0,1,1.000000,1.000000,3.000000,2.000000,0,5,5,0
7,7,1928,1,2,2/17/29,NaN,Athletic Madrid,Donostia,0:3,0,...,1,0,3.000000,2.000000,1.000000,1.000000,0,6,7,0
8,8,1928,1,2,2/17/29,NaN,Real Unión,Racing,3:1,3,...,-1,-1,2.000000,3.000000,0.000000,2.000000,0,7,8,0
9,9,1928,1,2,2/17/29,NaN,Catalunya,Arenas Club,5:2,5,...,-1,-1,0.000000,5.000000,2.000000,3.000000,0,8,4,0


In [67]:
features_final = features_2.drop(['date','time','year_time'])

KeyError: "['date' 'time' 'year_time'] not found in axis"

In [68]:
features_2.to_csv('features_new.csv')